In [1]:
import os
import random
from tqdm import tqdm
from datetime import datetime
from collections import deque

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
from torch.autograd import Variable, grad

import torchvision.transforms as transforms
# import torchvision.transforms.v2 as v2
# import albumentations as A
# from albumentations.pytorch import ToTensorV2

from torch.utils.data import Subset
from torch.utils.data import DataLoader, Dataset

In [2]:
debug_mode = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

input_root_dir = "../data/"
output_root_dir = "./"

unlabeled_image_dir = input_root_dir + "unlabeled/"
labeled_image_dir = input_root_dir + "train/image/"
labeled_mask_dir = input_root_dir + "train/label/"
test_image_dir = input_root_dir + "test/image/"

date_time = datetime.now().strftime("%m_%d_%H_%M_%S")
if debug_mode:
    date_time = "today"
    
current_output_dir = output_root_dir + f"{date_time}/"
os.makedirs(current_output_dir, exist_ok=True)

weights_save_dir = current_output_dir + "model_weights/"
os.makedirs(weights_save_dir, exist_ok=True)

test_output_dir = current_output_dir + "outputs/"

Device: cuda


In [3]:
class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir=None, transform=None, mask_transform=None, crop_size=(512, 512)):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        
        self.transform = transform
        self.mask_transform = mask_transform
        
        self.image_paths = sorted(os.listdir(image_dir))
        self.mask_paths = sorted(os.listdir(mask_dir)) if mask_dir else None
        self.crop_size = crop_size

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(image_path).convert("RGB")
        
        if self.mask_paths:
            mask_path = os.path.join(self.mask_dir, self.mask_paths[idx])
            mask = np.load(mask_path)  # Load mask from NPY file

            # Convert integer mask to one-hot encoded tensor
            mask_onehot = np.zeros((*mask.shape, 3))  # For 3 classes
            for i in range(1, 4):  # For classes 1, 2, 3
                mask_onehot[..., i-1][mask == i] = 1

            mask = Image.fromarray(mask_onehot.astype(np.uint8))  # Convert to PIL Image for transformations
            
            
            # Random horizontal flip
            if random.random() > 0.5:
                image = transforms.functional.hflip(image)
                mask  = transforms.functional.hflip(mask)

            # Random rotation
            angle = random.uniform(-180, 180)
            image = transforms.functional.rotate(image, angle)
            mask = transforms.functional.rotate(mask, angle)

            # Random crop both image and mask
            i, j, h, w = transforms.RandomCrop.get_params(image, output_size=self.crop_size)
            image = transforms.functional.crop(image, i, j, h, w)
            mask  = transforms.functional.crop(mask, i, j, h, w)

            image = self.transform(image)
            mask  = self.mask_transform(mask)

            return image, mask
        else:
            # Random horizontal flip
            if random.random() > 0.5:
                image = transforms.functional.hflip(image)

            # Random rotation
            angle = random.uniform(-180, 180)
            image = transforms.functional.rotate(image, angle)

            # Random crop the image
            i, j, h, w = transforms.RandomCrop.get_params(image, output_size=self.crop_size)
            image = transforms.functional.crop(image, i, j, h, w)

            image = self.transform(image)
            
            return image, image_path
        
        
image_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

mask_transform = transforms.Compose([
    transforms.ToTensor(),
])


unlabeled_dataset = CustomDataset(image_dir=unlabeled_image_dir, transform=image_transform)
labeled_dataset = CustomDataset(image_dir=labeled_image_dir, mask_dir=labeled_mask_dir, transform=image_transform, mask_transform=mask_transform)
test_dataset = CustomDataset(image_dir=test_image_dir, transform=image_transform)

# Split labeled dataset into train and validation sets
train_indices, val_indices = train_test_split(list(range(len(labeled_dataset))), test_size=0.3, random_state=42)
train_dataset = Subset(labeled_dataset, train_indices)
val_dataset = Subset(labeled_dataset, val_indices)

unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=24, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [4]:
# # sanity check for first sample
# itr = 0
# for i, (images, img_path) in enumerate(unlabeled_loader):
#     normalized_image = images[0].permute(1, 2, 0).cpu().numpy()  # Permute to (H, W, C) format
    
#     original = (normalized_image * 0.229 + 0.485).clip(0, 1)  # Reverse normalization
#     noisy_image = normalized_image + 0.1 * np.random.randn(*normalized_image.shape)
    
#     fig, axes = plt.subplots(1, 3, figsize=(15, 5))
#     axes[0].imshow(original)
#     axes[0].set_title('Original')
#     axes[0].axis('off')
    
#     axes[1].imshow(normalized_image)
#     axes[1].set_title('Normalized')
#     axes[1].axis('off')
    
#     axes[2].imshow(noisy_image)
#     axes[2].set_title('Noisy')
#     axes[2].axis('off')

#     plt.show()

#     itr += 1
#     if itr > 5:
#         break

**Labels:** \
Vitrinite: 1 (red channel) \
Inertinite: 2 (green channel) \
Liptinite: 3 (blue channel)

In [5]:
# # sanity check for first sample
# itr = 0
# for i, (images, masks) in enumerate(train_loader):
#     normalized_image = images[0].cpu().numpy().transpose((1, 2, 0))
#     mask = masks[0].cpu().numpy().transpose((1, 2, 0))

#     original = (normalized_image * 0.229 + 0.485).clip(0, 1)
#     mask *= 255

#     fig, axes = plt.subplots(1, 3, figsize=(15, 5))
#     axes[0].imshow(original)
#     axes[0].set_title('Original Image')
#     axes[0].axis('off')
    
#     axes[1].imshow(normalized_image)
#     axes[1].set_title('Normalized')
#     axes[1].axis('off')

#     axes[2].imshow(mask, cmap='gray')
#     axes[2].set_title('True Mask')
#     axes[2].axis('off')

#     plt.show()
    
#     itr += 1
#     if itr > 5:
#         break

In [6]:
class Generator(nn.Module):
    def __init__(self, latent_dim=1024):
        super(Generator, self).__init__()
        
        # Initial size before upsampling
        self.init_size = 4  
        # Fully connected layer to reshape the latent vector to a feature map
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 512 * self.init_size * self.init_size))

        # Convolutional blocks for upsampling and feature transformation
        self.conv_blocks = nn.Sequential(
            # Block 1: Input size -> (512, 4, 4)
            nn.Conv2d(512, 512, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),  # (512, 8, 8)

            # Block 2: Input size -> (512, 8, 8)
            nn.Conv2d(512, 512, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),  # (512, 16, 16)

            # Block 3: Input size -> (512, 16, 16)
            nn.Conv2d(512, 512, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),  # (512, 32, 32)

            # Block 4: Input size -> (512, 32, 32)
            nn.Conv2d(512, 512, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),  # (512, 64, 64)

            # Block 5: Input size -> (512, 64, 64)
            nn.Conv2d(512, 256, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),  # (256, 128, 128)

            # Block 6: Input size -> (256, 128, 128)
            nn.Conv2d(256, 128, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),  # (128, 256, 256)

            # Block 7: Input size -> (128, 256, 256)
            nn.Conv2d(128, 64, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),  # (64, 512, 512)

            # Block 8: Input size -> (64, 512, 512)
            nn.Conv2d(64, 32, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            # Final upsample to get (32, 1024, 1024) if required but skipped here
            # nn.Upsample(scale_factor=2),

            # Block 9: Input size -> (32, 512, 512)
            nn.Conv2d(32, 16, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(16, 16, 3, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            # Output layer: Convert to 3 channels (RGB)
            nn.Conv2d(16, 3, 1, padding=0),
            nn.Tanh()  # Use Tanh to normalize the output between -1 and 1
        )

    def forward(self, z):
        # Flatten the input tensor to shape (batch_size, latent_dim)
        z = z.view(z.size(0), -1)
        # Transform the latent vector into the initial feature map
        out = self.l1(z)
        out = out.view(out.shape[0], 512, self.init_size, self.init_size)
        # Pass through the convolutional blocks
        img = self.conv_blocks(out)
        return img


In [7]:
class MinibatchStdDev(nn.Module):
    def forward(self, x):
        batch_size, _, height, width = x.size()
        stddev = torch.std(x, dim=0, keepdim=True)
        stddev_mean = stddev.mean().view(1, 1, 1, 1)
        stddev_mean = stddev_mean.expand(batch_size, 1, height, width)
        return torch.cat([x, stddev_mean], 1)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def conv_block(in_channels, out_channels, kernel_size=3, stride=1, padding=1, use_leaky_relu=True):
            layers = [
                nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
                nn.BatchNorm2d(out_channels)
            ]
            if use_leaky_relu:
                layers.append(nn.LeakyReLU(0.2, inplace=True))
            else:
                layers.append(nn.ReLU(inplace=True))
            return nn.Sequential(*layers)

        self.conv_blocks = nn.Sequential(
            conv_block(3, 16, kernel_size=1, stride=1, padding=0),  # 512x512
            conv_block(16, 16),  # 512x512
            conv_block(16, 32),  # 512x512
            nn.AvgPool2d(2),  # 256x256
            conv_block(32, 32),  # 256x256
            conv_block(32, 64),  # 256x256
            nn.AvgPool2d(2),  # 128x128
            conv_block(64, 64),  # 128x128
            conv_block(64, 128),  # 128x128
            nn.AvgPool2d(2),  # 64x64
            conv_block(128, 128),  # 64x64
            conv_block(128, 256),  # 64x64
            nn.AvgPool2d(2),  # 32x32
            conv_block(256, 256),  # 32x32
            conv_block(256, 512),  # 32x32
            nn.AvgPool2d(2),  # 16x16
            conv_block(512, 512),  # 16x16
            conv_block(512, 512),  # 16x16
            nn.AvgPool2d(2),  # 8x8
            conv_block(512, 512),  # 8x8
            conv_block(512, 512),  # 8x8
            nn.AvgPool2d(2),  # 4x4
            MinibatchStdDev(),  # 513x4x4
            conv_block(513, 512),  # 4x4
            nn.Conv2d(512, 512, kernel_size=4, stride=1, padding=0, bias=False),  # 1x1
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.fc = nn.Sequential(
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        out = self.conv_blocks(img)
        out = out.view(out.size(0), -1)
        validity = self.fc(out)
        return validity

In [8]:
netG = Generator()
netD = Discriminator()

netG.to(device)
netD.to(device)

# Define training parameters
num_epochs = 200
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define optimizer parameters
lr = 0.001
beta1 = 0.0
beta2 = 0.99
epsilon = 1e-8
n_critic = 1

# Initialize optimizers
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, beta2), eps=epsilon)
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2), eps=epsilon)

# To track the moving average of the generator and discriminator losses
loss_window_G = deque(maxlen=5)
loss_window_D = deque(maxlen=5)

def compute_gradient_penalty(D, real_samples, fake_samples):
    alpha = torch.rand(real_samples.size(0), 1, 1, 1, device=real_samples.device)
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(torch.ones(real_samples.size(0), 1, device=real_samples.device), requires_grad=False)
    
    gradients = grad(outputs=d_interpolates, inputs=interpolates,
                     grad_outputs=fake, create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    
    return gradient_penalty

# Training loop
for epoch in range(num_epochs):
    epoch_lossG = 0
    epoch_lossD = 0
    loader = tqdm(enumerate(unlabeled_loader), total=len(unlabeled_loader), desc=f'Epoch [{epoch+1}/{num_epochs}]') if debug_mode else enumerate(unlabeled_loader)
    
    for i, (images, img_path) in loader:
        # Move real masks to device
#         real_masks = masks.to(device)
        real_images = images.to(device)
        batch_size = real_images.size(0)

        # Create labels
        real_labels = torch.full((batch_size,), 1.0, device=device)
        fake_labels = torch.full((batch_size,), 0.0, device=device)

        ##########################
        # Train Discriminator
        ##########################
        netD.zero_grad()

        # Forward pass real masks through Discriminator
#         output_real = netD(real_masks).view(-1)
        output_real = netD(real_images).view(-1)
        lossD_real = -torch.mean(output_real)
        lossD_real.backward()

        # Generate fake masks with Generator
        seed = torch.rand(batch_size, 1024, 1, 1, device=device)
#         fake_masks = netG(seed).detach()
        fake_images = netG(seed).detach()

        # Forward pass fake masks through Discriminator
#         output_fake = netD(fake_masks).view(-1)
        output_fake = netD(fake_images).view(-1)
        lossD_fake = torch.mean(output_fake)
        lossD_fake.backward()

        # Compute gradient penalty
#         gradient_penalty = compute_gradient_penalty(netD, real_masks.data, fake_masks.data)
        gradient_penalty = compute_gradient_penalty(netD, real_images.data, fake_images.data)
        gradient_penalty.backward()

        # Update Discriminator
        optimizerD.step()

        ##########################
        # Train Generator
        ##########################
        netG.zero_grad()

        # Forward pass fake masks through Discriminator
#         output = netD(fake_masks).view(-1)
        output = netD(fake_images).view(-1)
        lossG = -torch.mean(output)
        lossG.backward()

        # Update Generator
        optimizerG.step()

        epoch_lossG += lossG.item()

        epoch_lossD += (lossD_real.item() + lossD_fake.item())

        # Print training stats
        if debug_mode:
            loader.set_postfix(D_Loss=lossD_real.item() + lossD_fake.item(), G_Loss=lossG.item())
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i}/{len(unlabeled_loader)}], '
        f'D Loss: {lossD_real.item() + lossD_fake.item():.4f}, G Loss: {lossG.item():.4f}')
    
    avg_lossG = epoch_lossG / len(unlabeled_loader)
    avg_lossD = epoch_lossD / len(unlabeled_loader)
    loss_window_G.append(avg_lossG)
    loss_window_D.append(avg_lossD)
    
    # Step the learning rate schedulers
    # Note: Learning rate schedulers are not used in the original code, so they are omitted here
    
    # Save model weights if the current average loss is less than the moving average
    if len(loss_window_G) == loss_window_G.maxlen and len(loss_window_D) == loss_window_D.maxlen:
        moving_avg_lossG = np.mean(loss_window_G)
        moving_avg_lossD = np.mean(loss_window_D)
        if avg_lossG < moving_avg_lossG and avg_lossD < moving_avg_lossD:
            torch.save(netG.state_dict(), weights_save_dir + 'netG_best.pth')
            torch.save(netD.state_dict(), weights_save_dir + 'netD_best.pth')
            print(f'Saved models at epoch {epoch+1}, G Loss: {avg_lossG:.4f}, D Loss: {avg_lossD:.4f}, '
                  f'Moving Avg G Loss: {moving_avg_lossG:.4f}, Moving Avg D Loss: {moving_avg_lossD:.4f}')

print("Training finished!")

Epoch [1/200]: 100%|██████████| 360/360 [15:46<00:00,  2.63s/it, D_Loss=2.38e-7, G_Loss=-1]    


Epoch [1/200], Step [359/360], D Loss: 0.0000, G Loss: -1.0000


Epoch [2/200]: 100%|██████████| 360/360 [15:44<00:00,  2.62s/it, D_Loss=1.19e-7, G_Loss=-1]


Epoch [2/200], Step [359/360], D Loss: 0.0000, G Loss: -1.0000


Epoch [3/200]: 100%|██████████| 360/360 [15:40<00:00,  2.61s/it, D_Loss=0, G_Loss=-1]       


...

Epoch [26/200]: 100%|██████████| 360/360 [16:02<00:00,  2.67s/it, D_Loss=0, G_Loss=-1]


Epoch [26/200], Step [359/360], D Loss: 0.0000, G Loss: -1.0000


Epoch [27/200]: 100%|██████████| 360/360 [16:03<00:00,  2.68s/it, D_Loss=0, G_Loss=-1]


Epoch [27/200], Step [359/360], D Loss: 0.0000, G Loss: -1.0000


Epoch [28/200]:  49%|████▉     | 178/360 [07:56<08:08,  2.68s/it, D_Loss=0, G_Loss=-1]